<a href="https://colab.research.google.com/github/divinadira/Case-Study-3-Data-Mining_Association-Rules_Group-H/blob/main/Case-Study-3_Group-H_Association-Rule.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Group H**

1.	Divina Fatriandira (1906299465)
2.	Eslim Suyangsu Rohmanullah (1906299471)
3.	Muhammad Fabian Alkautsar (2006482653)
4.	M. Dzaky Jaya (1806193640)
5.	Rosyda Hanavania (1906299566)
6.	Timothy Devan Chandra (1906375726)
7.	Zalfa Nurfadhilah Haris (1906375726)



# Kasus

* AlfaIndo adalah sebuah toko waralaba yang memiliki beberapa cabang di kota-kota besar di Indonesia seperti Jakarta, Depok, Surabaya, dan Medan.
* Kami adalah seorang Data Analyst yang bekerja di kantor pusat AlfaIndo di Jakarta.
* Pasca masa pandemi **Toko AlfaIndo cabang UI Depok mengalami penurunan laba yang signifikan**.
* Kami ditugaskan oleh pimpinan untuk membantu toko AlfaIndo cabang UI Depok untuk bangkit dari keterpurukannya.

# Tujuan

Membuat rekomendasi dari data transaksi agar membantu toko AlfaIndo cabang UI Depok untuk bangkit dari penurunan laba


# Modul dan Packages yang Digunakan

In [46]:
# IMPORT MODULE
try:
    import google.colab as gc_
    print("Running the code in Google Colab.", gc_) #IN_COLAB = True
    print("Installing required Module, please wait ... ")
    !pip install mlxtend 
    !pip install pycaret
except:
    print("Running the code locally, make sure to install required modules.")

Running the code in Google Colab. <module 'google.colab' from '/usr/local/lib/python3.7/dist-packages/google/colab/__init__.py'>
Installing required Module, please wait ... 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [47]:
import warnings; warnings.simplefilter('ignore')
import pandas as pd, matplotlib.pyplot as plt, seaborn as sns
from itertools import combinations
from collections import Counter
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from pycaret.arules import *

%matplotlib inline
plt.style.use('bmh'); sns.set()

# Penginputan Data
Data transaksi yang diberikan adalah dalam format excel. Kami mengubah format file tersebut menjadi csv karena scalability dari format csv lebih luas dibandingkan dengan format excel.

In [48]:
df = pd.read_csv('https://raw.githubusercontent.com/divinadira/Case-Study-3-Data-Mining_Association-Rules_Group-H/main/Kasus-AR-MBA-Ganjil-2022.csv', sep=";")

**Catatan**
Data transaksi yang diberikan telah kami rapikan menjadi transaksi peritem (satu baris terdiri dari pembelian satu merek produk), tanpa indexing. Hal ini kami lakukan untuk mempermudah proses Data Cleansing.

Selain itu, kami membagi dan memisahkan data `Tanggal` menjadi variabel `Tanggal` dan `Hari`. Hal ini dilakukan agar kami bisa menganalisis data berdasarkan hari terjadinya transaksi

# ***PREPROCESSING DATA***

## Informasi Data

In [49]:
# Cuplikan data teratas
df.head()

,ID Transaksi,Tanggal,Hari,Konsumen,ID Produk,Nama Produk,Merek,Jumlah
0,1,01/07/2022,Friday,Mr. A,1,Roti,Sri Roti,2
1,1,01/07/2022,Friday,Mr. A,2,Air Minum,Aquos,1
2,1,01/07/2022,Friday,Mr. A,11,Mie Instan,Indomi,2
3,1,01/07/2022,Friday,Mr. A,12,Mie Instan,Mi Sedap,1
4,2,01/07/2022,Friday,Mr. B,2,Air Minum,Aquos,1


In [50]:
# Cuplikan data terbawah
df.tail()

,ID Transaksi,Tanggal,Hari,Konsumen,ID Produk,Nama Produk,Merek,Jumlah
51,17,05/09/2022,Monday,Mr. X,2,Air Minum,Aquos,1
52,17,05/09/2022,Monday,Mr. X,4,Sabun,Lifboy,1
53,17,05/09/2022,Monday,Mr. X,6,Snack,Chiki,2
54,18,02/10/2022,Sunday,Mr. B,1,Roti,Sri Roti,2
55,18,02/10/2022,Sunday,Mr. B,10,Air Minum,Sprit,2


In [51]:
# Informasi data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ID Transaksi  56 non-null     int64 
 1   Tanggal       56 non-null     object
 2   Hari          56 non-null     object
 3   Konsumen      56 non-null     object
 4   ID Produk     56 non-null     int64 
 5   Nama Produk   56 non-null     object
 6   Merek         56 non-null     object
 7   Jumlah        56 non-null     int64 
dtypes: int64(3), object(5)
memory usage: 3.6+ KB


In [52]:
df.shape

(56, 8)

**Catatan**

Data terdiri dari 56 baris dan 8 kolom. Hal yang kami dapatkan dari hasil analisis awal di atas adalah:
* `ID Transaksi` memiliki tipe *int_64*, tipe data ini sudah sesuai dengan yang seharusnya
* `Tanggal` memiliki tipe *object*, data ini seharusnya bertipe *datetime64*
* `Hari` memiliki tipe *object*, data ini seharusnya bertipe *category*
* `Konsumen` memiliki tipe *object*, data ini seharusnya bertipe *category*
* `ID Produk` memiliki tipe *int_64*, tipe data ini sudah sesuai dengan yang seharusnya
* `Nama Produk` memiliki tipe *object*, data ini seharusnya bertipe *category*
* `Merek` memiliki tipe *object*, data ini seharusnya bertipe *category*
* `Jumlah` memiliki tipe *int_64*, tipe data ini sudah sesuai dengan yang seharusnya

In [53]:
df['Tanggal'] = df['Tanggal'].astype('datetime64')
df['Hari'] = df['Hari'].astype('category')
df['Konsumen'] = df['Konsumen'].astype('category')
df['Nama Produk'] = df['Nama Produk'].astype('category')
df['Merek'] = df['Merek'].astype('category')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   ID Transaksi  56 non-null     int64         
 1   Tanggal       56 non-null     datetime64[ns]
 2   Hari          56 non-null     category      
 3   Konsumen      56 non-null     category      
 4   ID Produk     56 non-null     int64         
 5   Nama Produk   56 non-null     category      
 6   Merek         56 non-null     category      
 7   Jumlah        56 non-null     int64         
dtypes: category(4), datetime64[ns](1), int64(3)
memory usage: 3.6 KB


In [54]:
print(df.isnull().sum())

ID Transaksi    0
Tanggal         0
Hari            0
Konsumen        0
ID Produk       0
Nama Produk     0
Merek           0
Jumlah          0
dtype: int64


**Catatan**

Tidak terdapat Missing Value dalam data

In [55]:
df.describe(include = 'all').transpose()

,count,unique,top,freq,first,last,mean,std,min,25%,50%,75%,max
ID Transaksi,56.0,NaN,NaN,NaN,NaT,NaT,8.660714,5.085726,1.0,4.0,9.0,12.25,18.0
Tanggal,56,11,2022-05-07 00:00:00,9,2022-01-07,2022-08-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Hari,56,7,Monday,10,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Konsumen,56,12,Mr. B,15,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ID Produk,56.0,NaN,NaN,NaN,NaT,NaT,6.25,3.475891,1.0,3.75,6.0,9.25,12.0
Nama Produk,56,7,Air Minum,11,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Merek,56,12,Chiki,7,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Jumlah,56.0,NaN,NaN,NaN,NaT,NaT,1.5,0.572078,1.0,1.0,1.0,2.0,3.0


**Catatan**

Dari hasil Deskriptif Data dan analisis kami, kami mendapat beberapa informasi berikut:
* Data yang diberikan sudah cukup rapi. Tidak ditemukan data Noise yang merupakan data tidak valid.
* `Tanggal` memiliki unique value 11. Hal ini sesuai dengan data yang ada, yaitu hanya mencatat transaksi dari 11 hari penjualan.
  
  Info tambahan: Tanggal paling banyak terjadi transaksi adalah tanggal 5 Juli 2022.
* `Hari` memiliki unique value 7. Hal ini sesuai dengan Data Understanding, yaitu hari dalam seminggu maksimal ada 7. Value untuk variabel ini akan dikeluarkan hasilnya pada tahap selanjutnya.

  Info tambahan: Hari yang paling sering terdapat transaksi adalah hari Senin

* `Konsumen` memiliki unique value 12. Hal ini sesuai dengan Data yang memunjukkan ada 12 konsumen yang transaksinya tercatat dalam data. Value untuk variabel ini akan dikeluarkan hasilnya pada tahap selanjutnya.
  
  Info tambahan: Konsumen yang paling sering melakukan transaksi adalah Mr. B

* `Nama Produk` memiliki unique value 7. Hal ini sesuai dengan Data Understanding yang memunjukkan terdapat 7 Nama Produk dalam Toko AlfaIndo. Value untuk variabel ini akan dikeluarkan hasilnya pada tahap selanjutnya.

* `Merek` memiliki unique value 12. Hal ini sesuai dengan Data Understanding yang memunjukkan terdapat 12 Merek dalam Toko AlfaIndo. Value untuk variabel ini akan dikeluarkan hasilnya pada tahap selanjutnya.



In [56]:
# Value dari variabel Hari
df['Hari'].dtypes

CategoricalDtype(categories=['Friday', 'Monday', 'Saturday', 'Sunday', 'Thursday',
                  'Tuesday', 'Wednesday'],
, ordered=False)

In [57]:
# Value dari variabel Konsumen
df['Konsumen'].dtypes

CategoricalDtype(categories=['Mr. A', 'Mr. B', 'Mr. C', 'Mr. D', 'Mr. F', 'Mr. G',
                  'Mr. J', 'Mr. P', 'Mr. T', 'Mr. X', 'Mr. Z', 'Mr.Y'],
, ordered=False)

In [58]:
# Value dari variabel Nama Produk
df['Nama Produk'].dtypes

CategoricalDtype(categories=['Air Minum', 'Mie Instan', 'Rokok', 'Roti', 'Sabun',
                  'Shampoo', 'Snack'],
, ordered=False)

In [59]:
# Value dari variabel Merek
df['Merek'].dtypes

CategoricalDtype(categories=['Aquos', 'Chiki', 'Citata', 'Clean', 'Indomi', 'Lifboy',
                  'Luks', 'Malioboro', 'Mi Sedap', 'Prambanan', 'Sprit',
                  'Sri Roti'],
, ordered=False)

# ***EXPLORATORY DATA ANALYSIS***

Kami akan melakukan EDA dengan Visualisasi Data yang kami bagi berdasarkan variabel-variabel


## Total Penjualan berdasarkan Tanggal
![picture](https://drive.google.com/uc?export=view&id=1mjYSxMWtwbytAJswL9ZQjcIYRtHxB7yh)

Dari grafik terlihat bahwa ternyata terdapat data transaksi yang tidak lengkap sejak 9 Juli 2015. Maka, kami mengasumsikan bahwa sejak tanggal 9 Juli 2015 toko AlfaIndo tidak ada transaksi hingga 1 Agustus 2015. Kemudian, untuk bulan Agustus, September, dan Oktober transaksi hanya terjadi 1 hari dalam sebulan, yaitu pada tanggal 2 Agustus, 5 September, dan 2 Oktober. 

In [60]:
df.tail(8)

,ID Transaksi,Tanggal,Hari,Konsumen,ID Produk,Nama Produk,Merek,Jumlah
48,15,2022-08-07,Friday,Mr. F,9,Rokok,Malioboro,1
49,16,2022-02-08,Tuesday,Mr. J,2,Air Minum,Aquos,2
50,17,2022-05-09,Monday,Mr. X,1,Roti,Sri Roti,2
51,17,2022-05-09,Monday,Mr. X,2,Air Minum,Aquos,1
52,17,2022-05-09,Monday,Mr. X,4,Sabun,Lifboy,1
53,17,2022-05-09,Monday,Mr. X,6,Snack,Chiki,2
54,18,2022-02-10,Sunday,Mr. B,1,Roti,Sri Roti,2
55,18,2022-02-10,Sunday,Mr. B,10,Air Minum,Sprit,2


Pada tanggal 2 Agustus 2015 terlihat pada data bahwa terdapat pelanggan baru yaitu Mr. J yang melakukan transaksi, tetapi tidak melakukan transaksi kedua. Kemudian, pada tanggal 5 September 2015 dan 2 Oktober 2015 Mr. X dan Mr. B yang merupakan pelanggan lama, melakukan transaksi kembali di AlfaIndo. 

Kami merekomendasikan Toko AlfaIndo cabang UI Depok melakukan promosi untuk transaksi kedua. Hal ini dilakukan agar pelanggan baru, misalnya Mr. J, dapat terus melakukan transaksi selanjutnya, bukan hanya satu kali transaksi di Toko AlfaIndo.

## Total Penjualan berdasarkan Hari

![picture](https://drive.google.com/uc?export=view&id=1_IowalqB7BAp0haVNtj_bm9rqOrZfVD8)

Dapat dilihat pada grafik bahwa penjualan terbanyak terdapat pada hari Selasa. Sedangkan, untuk penjualan paling sedikit terdapat pada hari Rabu dan Kamis. Dari hal tersebut, kami dapat menyarankan kepada Alfaindo untuk memberlakukan promo yang lebih banyak pada hari Rabu dan Kamis. Rekomendasi lain adalah menambah stok lebih pada hari Selasa, sehingga penjualan tetap banyak pada hari tersebut

## Total Penjualan berdasarkan Nama Produk

![picture](https://drive.google.com/uc?export=view&id=1bm7iJmql64OL5L0V-wI9wG8FtbT5NuIb)

Pada grafik di atas dapat dilihat bahwa jenis produk Air Minum, Roti, dan Snack memiliki jumlah penjualan paling banyak diantara produk yang lain. Sedangkan, jumlah penjualan paling sedikit ada pada jenis produk Rokok. Untuk itu, kami menyarankan agar Alfaindo memperbanyak variasi dan stock produk Air Minum, Roti, dan Snack.

Selanjutnya akan divisualisasikan penjualan untuk setiap Merek pada nama produk. Visualisasi untuk produk Rokok dan Shampoo tidak ditampilkan karena hanya memiliki 1 merek.

## Penjualan Air Minum

![picture](https://drive.google.com/uc?export=view&id=1n27jHctQu6eczLXfWwrNYhZH5C3Z8Fj5)

Dari grafik di atas, dapat dilihat bahwa dari 2 merek Air Minum yang dijual Alfaindo, produk yang paling laku terjual adalah merek Sprit. Dari hal tersebut, kami menyarankan agar Alfaindo memperbanyak variasi/stock Air Minum Sprit dan memperbanyak promo untuk produk Air Minum Aquos.

## Penjualan Mie Instan
![picture](https://drive.google.com/uc?export=view&id=18mATDr54258UcITZGLwPKcW8EWT4AmPA)

Dari grafik di atas, dapat dilihat bahwa dari 2 merek Mie Instan yang dijual Alfaindo, produk yang paling laku terjual adalah merek Indomi. Dari hal tersebut, kami menyarankan agar Alfaindo memperbanyak variasi/stock Mie Instan Indomi dan memperbanyak promo untuk produk Mie Instan Mi Sedap.

## Penjualan Roti
![picture](https://drive.google.com/uc?export=view&id=1fHDXoz7WYoRVidVusBmnqZWb4d36S_3L)

Dari grafik di atas, dapat dilihat bahwa dari 2 merek Roti yang dijual Alfaindo, produk yang paling laku terjual adalah merek Sri Roti. Dari hal tersebut, kami menyarankan agar Alfaindo memperbanyak variasi/stock Roti Sri Roti dan memperbanyak promo untuk produk Roti Prambanan.

## Penjualan Sabun
![picture](https://drive.google.com/uc?export=view&id=115I2HVKHJBauNOCsDDpM8Y0McLNQkwF2)

Dari grafik di atas, dapat dilihat bahwa dari 2 merek Sabun yang dijual Alfaindo, produk yang paling laku terjual adalah merek Lifboy. Dari hal tersebut, kami menyarankan agar Alfaindo memperbanyak variasi/stock Sabun Lifboy dan memperbanyak promo untuk produk Sabun Luks.

## Penjualan Snack
![picture](https://drive.google.com/uc?export=view&id=1jcHPNudqUX4X41rbhQZ7BoVhJxOXfolb)

Dari grafik di atas, dapat dilihat bahwa dari 2 merek Snack yang dijual Alfaindo, produk yang paling laku terjual adalah merek Chiki. Dari hal tersebut, kami menyarankan agar Alfaindo memperbanyak variasi/stock Snack Chiki dan memperbanyak promo untuk produk Snack Citata.

# ***MARKET BASKET ANALYSIS***

## Mengubah format data

Pengubahan format data dilakukan untuk menyamakan dengan format yang dapat diterima module MLXTend

In [61]:
import warnings; warnings.simplefilter('ignore')
import pandas as pd, numpy as np

'Done'

'Done'

In [62]:
# Memberikan Index Data
index_transaksi = df["ID Transaksi"]
index_idproduk = df["ID Produk"]

df = df.set_index([index_transaksi,index_idproduk])
df["Jumlah"]

ID Transaksi  ID Produk
1             1            2
              2            1
              11           2
              12           1
2             2            1
              6            2
              3            1
              5            1
3             4            1
              5            1
              9            1
4             1            2
              10           2
              8            3
              7            1
              12           2
5             6            1
              10           2
6             11           2
              4            1
              5            1
7             10           1
              9            1
8             6            1
              7            1
              5            1
              11           3
9             1            2
              2            1
              4            1
              6            2
              9            1
              11           2
10            3    

**Catatan**

Diberikan indexing pada `ID Transaksi` dan `ID Produk`. Hasil dari indexing ini kami dapat melihat jumlah transaksi yang terjadi per ID Transaksi dan per ID Produk

In [63]:
# Kami akan mengambil beberapa kolom saja untuk dapat diposes dalam Market Basket Analysis
# Ini akan melihat persebaran produk yang akan dijual 

df["Jumlah"].unstack()
df2 = df["Jumlah"].unstack()
df2

ID Produk,1,2,3,4,5,6,7,8,9,10,11,12
ID Transaksi,,,,,,,,,,,,
1,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0
2,NaN,1.0,1.0,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN
4,2.0,NaN,NaN,NaN,NaN,NaN,1.0,3.0,NaN,2.0,NaN,2.0
5,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,2.0,NaN,NaN
6,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN
8,NaN,NaN,NaN,NaN,1.0,1.0,1.0,NaN,NaN,NaN,3.0,NaN
9,2.0,1.0,NaN,1.0,NaN,2.0,NaN,NaN,1.0,NaN,2.0,NaN


In [64]:
#Akan replace NaN

import numpy as np
df2.replace(np.NaN, 0,inplace = True)
df2 = df2.astype("int32")
df2

ID Produk,1,2,3,4,5,6,7,8,9,10,11,12
ID Transaksi,,,,,,,,,,,,
1,2,1,0,0,0,0,0,0,0,0,2,1
2,0,1,1,0,1,2,0,0,0,0,0,0
3,0,0,0,1,1,0,0,0,1,0,0,0
4,2,0,0,0,0,0,1,3,0,2,0,2
5,0,0,0,0,0,1,0,0,0,2,0,0
6,0,0,0,1,1,0,0,0,0,0,2,0
7,0,0,0,0,0,0,0,0,1,1,0,0
8,0,0,0,0,1,1,1,0,0,0,3,0
9,2,1,0,1,0,2,0,0,1,0,2,0


**Catatan**

Format data telah diubah ke dalam bentuk Matriks Sparse yang dapat diterima oleh modul MLXTend.

Index yang terbentuk adalah `ID Transaksi` dan kolom yang terbentuk adalah `ID Produk`. Isi dari matriksnya adalah jumlah transaksi yang terjadi

In [65]:
#Rename ID Product dengan merk
df2.rename(columns = {1 : "Sri Roti",
2 : "Aquos",
3 : "Prambanan",
4 : "Lifboy",
5 : "Clean",
6 : "Chiki",
7 : "Luks",
8 : "Citata",
9 : "Malioboro",
10 : "Sprit",
11 : "Indomi",
12 :"Mi Sedap"},inplace = True)

In [66]:
df2.replace({2 : 1,
3 : 1}, inplace = True)

**Catatan**

Telah dilakukan pengubahan isi value dengan 0 dan 1. Hal ini dilakukan agar matriks dapat diolah dengan MLXTend

## Penggunaan MLXTend

### Menemukan Itemsets yang Paling Sering Dibeli

In [67]:
frequent_itemsets = apriori(df2, min_support=0.07, use_colnames=True)
frequent_itemsets.sort_values(by='support', ascending=False, na_position='last', inplace = True)
frequent_itemsets

,support,itemsets
5,0.388889,(Chiki)
0,0.333333,(Sri Roti)
4,0.333333,(Clean)
9,0.333333,(Sprit)
3,0.277778,(Lifboy)
8,0.277778,(Malioboro)
10,0.277778,(Indomi)
1,0.277778,(Aquos)
16,0.166667,"(Sprit, Sri Roti)"
27,0.166667,"(Chiki, Clean)"


**Catatan**

Dari hasil di atas, terlihat bahwa penjualan paling terdapat pada produk Chiki, kemudian disusul Sri Roti dan Clean. Sementara penjualan yang paling rendah terdapat pada produk Citata

### Menemukan Association Rules dari Data

In [68]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules.sort_values(by='lift', ascending=False, na_position='last', inplace = True)
rules.head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
97,"(Aquos, Lifboy)","(Chiki, Sri Roti)",0.111111,0.111111,0.111111,1.000000,9.0,0.098765,inf
96,"(Chiki, Sri Roti)","(Aquos, Lifboy)",0.111111,0.111111,0.111111,1.000000,9.0,0.098765,inf
95,"(Chiki, Lifboy)","(Aquos, Sri Roti)",0.111111,0.166667,0.111111,1.000000,6.0,0.092593,inf
63,"(Sri Roti, Sprit)",(Citata),0.166667,0.111111,0.111111,0.666667,6.0,0.092593,2.666667
66,(Citata),"(Sri Roti, Sprit)",0.111111,0.166667,0.111111,1.000000,6.0,0.092593,inf
99,"(Lifboy, Sri Roti)","(Chiki, Aquos)",0.111111,0.166667,0.111111,1.000000,6.0,0.092593,inf
98,"(Aquos, Sri Roti)","(Chiki, Lifboy)",0.166667,0.111111,0.111111,0.666667,6.0,0.092593,2.666667
94,"(Chiki, Aquos)","(Lifboy, Sri Roti)",0.166667,0.111111,0.111111,0.666667,6.0,0.092593,2.666667
51,"(Chiki, Clean)",(Luks),0.166667,0.166667,0.111111,0.666667,4.0,0.083333,2.500000
54,(Luks),"(Chiki, Clean)",0.166667,0.166667,0.111111,0.666667,4.0,0.083333,2.500000


In [69]:
rules.tail(5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
88,(Lifboy),(Sri Roti),0.277778,0.333333,0.111111,0.400000,1.200000,0.018519,1.111111
77,(Lifboy),(Chiki),0.277778,0.388889,0.111111,0.400000,1.028571,0.003086,1.018519
60,(Indomi),(Chiki),0.277778,0.388889,0.111111,0.400000,1.028571,0.003086,1.018519
61,(Chiki),(Indomi),0.388889,0.277778,0.111111,0.285714,1.028571,0.003086,1.011111
76,(Chiki),(Lifboy),0.388889,0.277778,0.111111,0.285714,1.028571,0.003086,1.011111


**Catatan**

Hasil Rule yang didapat dari data yang akan kami fokuskan antara lain:
* Lifboy & Aquos => Sri Roti & Chiki
* Sprit & Sri Roti => Citata

# ***PENUTUP***

## Rekomendasi untuk Toko AlfaIndo Cabang UI Depok

Terlihat hasil output di atas bahwa terdapat lift tertinggi untuk setiap pembelian Lifboy dan Aquos, pelanggan juga membeli Sri Roti dan Chiki. Selain itu, untuk pembelian Sprit dan Sri Roti, pelanggan juga membeli Citata.


Maka dari itu kami menyarankan untuk membuat promosi paket "Travel Pack" untuk pembelian Sprit, Sri Roti, dan Citata dengan tambahan Aquos dengan tujuan cross selling agar penjualan Aquos meningkat (berdasarkan nilai support yang rendah). Kemudian kami juga menyarankan untuk produk Sri Roti dan Lifboy untuk di jauhkan penempatannya (dengan asumsi toko berukuran kecil) agar pelanggan dapat melihat-lihat produk yang lainnya sehingga menaikkan penjualan.

Rekomendasi lainnya adalah menurunkan harga Sprit & Sri Roti. Hal ini dilakukan agar penjualan Citata akan naik karena berdasarkan Rules dikatakan bahwa Sprit & Sri Roti => Citata

